In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import math

In [ ]:
# --------------------- ECA模块定义 ---------------------
class ECALayer(nn.Module):
    def __init__(self, channels, gamma=2, b=1):
        super(ECALayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        # 动态计算卷积核大小
        kernel_size = int(abs((math.log(channels, 2) + b) / gamma))
        kernel_size = kernel_size if kernel_size % 2 else kernel_size + 1
        self.conv = nn.Conv1d(1, 1, kernel_size=kernel_size, padding=(kernel_size-1)//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # 通道注意力权重计算
        y = self.avg_pool(x)            # [B, C, 1, 1]
        y = y.squeeze(-1).transpose(1, 2)  # [B, 1, C]
        y = self.conv(y)                # 1D卷积捕获通道关系
        y = self.sigmoid(y)              # [B, 1, C]
        y = y.transpose(1, 2).unsqueeze(-1)  # [B, C, 1, 1]
        return x * y.expand_as(x)        # 特征图加权

In [ ]:
# --------------------- 精简版ResNet50+ECA ---------------------
class ECABottleneck(nn.Module):
    expansion = 4  # 扩展系数从1改为4

    def __init__(self, in_channels, out_channels, stride=1):
        super(ECABottleneck, self).__init__()
        # 1x1卷积降维
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        # 3x3卷积
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, 
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # 1x1卷积升维
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, 
                               kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        
        # ECA模块放在最后一个卷积之后
        self.eca = ECALayer(out_channels * self.expansion)

        # 捷径连接
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, 
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = torch.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = self.eca(out)  # 应用ECA注意力
        out += self.shortcut(x)
        return torch.relu(out)

class ECA_ResNet50(nn.Module):  
    def __init__(self, block, num_blocks, num_classes=10):
        super(ECA_ResNet50, self).__init__()
        self.in_channels = 64
        
        # 输入层
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        
        # 四个阶段（层数更多，通道数更大）
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)   # 输出256通道
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)  # 输出512通道
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)  # 输出1024通道
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)  # 输出2048通道
        
        # 输出层
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)  # 512*4=2048

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
